# Notebook: Train Model for a given Condition

## Packages

In [1]:
from ACD import aspect_category_labels_to_one_hot, CustomDatasetACD, preprocess_data_ACD, create_model_ACD, compute_metrics_ACD, get_trainer_ACD
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, precision_score, recall_score
from torch.utils.data import Dataset as TorchDataset
from helper import format_seconds_to_time_string
from transformers import DataCollatorWithPadding
from transformers import set_seed
from scipy.special import expit
import pandas as pd
import numpy as np
import datetime
import warnings
import random
import torch
import json
import time

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

device(type='mps')

## Parameters

In [3]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_term"
LLM_SAMPLING = "fixed"

## Settings (do not change!)

In [4]:
N_FOLDS = 5

In [5]:
SPLIT_LOOP = [0, 1, 2, 3, 4, 0, 1, 2, 3]

In [6]:
RANDOM_SEED = 43
random.seed(RANDOM_SEED)

In [7]:
ASPECT_CATEGORIES  = ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]
POLARITIES = ["POSITIVE", "NEUTRAL", "NEGATIVE"]

In [8]:
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)

In [9]:
set_seed(RANDOM_SEED)

## Code

### Load Dataset

In [10]:
# Load Real Dataset
splits_real = []
for i in range(N_FOLDS):
    with open(f'../03 dataset split/real/real_{i}.json', 'r') as json_datei:
        real_split = json.load(json_datei)[:N_REAL]
        splits_real.append(real_split)

In [11]:
# Load Synth Dataset
splits_synth = []
for i in range(N_FOLDS):
    with open(f'../04 llm synthesis/synth/{LLM_NAME}/{LLM_SAMPLING}/split_{i}.json', 'r') as json_datei:
        synth_split = json.load(json_datei)[:N_SYNTH]
        splits_synth.append(synth_split)

In [12]:
n_splits_map = {
    500: 1,
    1000: 2,
    2000: 4
}
n_splits_required_real = n_splits_map.get(N_REAL, 0)
n_splits_required_synth = n_splits_map.get(N_SYNTH, 0)
n_splits_required_real, n_splits_required_synth

(1, 0)

In [13]:
# Five indexes, each for one cross valdiation run
train_dataset = []
test_dataset = []

for i in range(N_FOLDS):
    test_data = splits_real[i]
    train_data = []
    
    if N_REAL > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_real]:
            for example in splits_real[split_idx]:
                train_data.append(example)
 
    if N_SYNTH > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_synth]:
            for example in splits_synth[split_idx]:
                train_data.append(example)
                
    random.shuffle(train_data)
    
    train_dataset.append(train_data)
    test_dataset.append(test_data)

### ACD Model

In [14]:
def train_ACD_model():
    results = {
        "LLM_NAME": LLM_NAME,
        "N_REAL": N_REAL,
        "N_SYNTH": N_SYNTH,
        "TARGET": TARGET,
        "LLM_SAMPLING": LLM_SAMPLING,
    }

    f1_micro_scores = []
    f1_macro_scores = []
    f1_weighted_scores = []
    accuracy_scores = []
    class_f1_scores = []
    loss = []
    hamming = []

    start_time = time.time()

    tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-large")
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    for cross_idx in range(N_FOLDS):
        # Load Data
        train_data = preprocess_data_ACD(train_dataset[cross_idx], tokenizer)
        test_data = preprocess_data_ACD(test_dataset[cross_idx], tokenizer)

        # Load Model
        model_ACD = create_model_ACD()

        # Train Model
        trainer = get_trainer_ACD(model_ACD, train_data, test_data, tokenizer)
        trainer.train()

        # Save Evaluation of Test Data
        eval_metrics = trainer.evaluate()

        # Save Metrics for fold
        f1_micro_scores.append(eval_metrics["eval_f1_micro"])
        f1_macro_scores.append(eval_metrics["eval_f1_macro"])
        f1_weighted_scores.append(eval_metrics["eval_f1_weighted"])
        accuracy_scores.append(eval_metrics["eval_accuracy"])
        class_f1_scores.append(eval_metrics["eval_class_f1_scores"])
        loss.append(eval_metrics["eval_loss"])
        hamming.append(eval_metrics["eval_hamming_loss"])

    runtime = time.time() - start_time

    results["loss"] = np.mean(loss)
    results["hamming"] = np.mean(hamming)
    results["accuracy"] = np.mean(accuracy_scores)
    results["f1_micro"] = np.mean(f1_micro_scores)
    results["f1_macro"] = np.mean(f1_macro_scores)
    results["f1_weighted"] = np.mean(f1_weighted_scores)
    results["runtime"] = runtime
    results["runtime_formatted"] = format_seconds_to_time_string(runtime)
    return results

if TARGET == "aspect_category":
   results = train_ACD_model()

### OTE Model

In [19]:
train_dataset[0][0]["text"], train_dataset[0][0]["tags"]


for cross_idx in range(N_FOLDS)[0:]:
    # Load Data
    train_data = train_dataset[cross_idx]
    test_data = test_dataset[cross_idx]

In [20]:
train_data[0]

{'tags': [{'end': 28,
   'start': 21,
   'tag_with_polarity': 'SERVICE-NEGATIVE',
   'tag_with_polarity_and_type': 'SERVICE-NEGATIVE-explicit',
   'text': 'Kellner',
   'type': 'label-explicit',
   'label': 'SERVICE',
   'polarity': 'NEGATIVE'},
  {'end': 118,
   'start': 110,
   'tag_with_polarity': 'FOOD-NEUTRAL',
   'tag_with_polarity_and_type': 'FOOD-NEUTRAL-explicit',
   'text': 'Getränke',
   'type': 'label-explicit',
   'label': 'FOOD',
   'polarity': 'NEUTRAL'}],
 'text': 'Irgendwann kam unser Kellner nicht mehr, wir mussten andere Kellner ansprechen, damit sie ihn holten, um noch Getränke bestellen zu können.',
 'two_or_more_sentences': False,
 'id': 'bc142ab2-b3c7-4b30-88d3-5d87f055c53d',
 'city': 'münchen',
 'date': '2022-11-09',
 'title': 'Super Essen, Service fragwürdig, Biere nicht voll.',
 'rating': 3.0,
 'review_id': 867815811,
 'page_index': 2,
 'author_name': '_BenitaFi-',
 'sentence_idx': 4,
 'language_code': 'de',
 'restaurant_id': 807825,
 'author_location': '',
 '

### Save Results

In [ ]:
results=[{}]

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

In [ ]:
df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results